# Instance Norm Operation

This notebook shows how to compute an instance Norm using the cuDNN python frontend.

$$\text{InstanceNorm(x)} = \frac{x - \mathbb{E}(x)}{\sqrt{Var(x)+\epsilon}}\cdot \gamma + \beta$$


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/cudnn-frontend/blob/main/samples/python/30_instancenorm.ipynb)

## Prerequisites and Setup
This notebook requires an NVIDIA GPU. If `nvidia-smi` fails, go to Runtime -> Change runtime type -> Hardware accelerator and confirm a GPU is selected.

In [ ]:
# get_ipython().system('nvidia-smi')

If running on Colab, you will need to install the cudnn python interface.

In [ ]:
# get_ipython().system('pip install nvidia-cudnn-cu12')
# get_ipython().system('pip install nvidia-cudnn-frontend')
# get_ipython().system('pip3 install --pre torch --index-url https://download.pytorch.org/whl/nightly/cu128')

## Overview

In the following, we are going to perform the instance norm forward pass. First we define the batch size, number of channels, spatial dimensions, and some other parameters:

In [ ]:
import cudnn
import torch

torch.manual_seed(1)
assert torch.cuda.is_available()

handle = cudnn.create_handle()

N, C, H, W = 16, 32, 64, 64
dtype = torch.float16
# Epsilon is a small number to prevent division by 0.
epsilon_value = 1e-5

## Using Wrapper

Below is how you can use the `Graph` wrapper to perform instance norm with the input tensors in PyTorch format.

In [ ]:
x_gpu = torch.randn(N, C, H, W, device="cuda", dtype=dtype).to(
    memory_format=torch.channels_last
)
scale_gpu = torch.randn(1, C, 1, 1, device="cuda").to(memory_format=torch.channels_last)
bias_gpu = torch.randn(1, C, 1, 1, device="cuda").to(memory_format=torch.channels_last)
eps_cpu = torch.full((1, 1, 1, 1), epsilon_value, dtype=torch.float32, device="cpu")

# forward pass of batchnorm using cuDNN graph
with cudnn.Graph(
    io_data_type=cudnn.data_type.HALF,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
    inputs=["in_fwd::input", "in_fwd::scale", "in_fwd::bias", "in_fwd::epsilon"],
    outputs=["out", "mean", "inv_var"],
) as fwd_graph:
    out, mean, inv_var = fwd_graph.instancenorm(
        name="in_fwd",
        norm_forward_phase=cudnn.norm_forward_phase.TRAINING,
        input=x_gpu,
        scale=scale_gpu,
        bias=bias_gpu,
        epsilon=eps_cpu,
    )
    out.set_output(True).set_data_type(dtype).set_name("out")
    mean.set_output(True).set_data_type(dtype).set_name("mean")
    inv_var.set_output(True).set_data_type(dtype).set_name("inv_var")

out_gpu, mean_gpu, inv_var_gpu = fwd_graph(
    x_gpu, scale_gpu, bias_gpu, eps_cpu, handle=handle
)

You can verify the correctness with PyTorch API:

In [ ]:
# PyTorch forward pass, and the stats of this batch
out_ref = torch.nn.functional.instance_norm(
    x_gpu,
    weight=scale_gpu.view(C),
    bias=bias_gpu.view(C),
)
mean_ref = torch.mean(x_gpu, dim=(2, 3), keepdim=True)
inv_var_ref = torch.rsqrt(torch.var(x_gpu, dim=(2, 3), keepdim=True) + epsilon_value)

torch.testing.assert_close(out_gpu, out_ref, atol=5e-3, rtol=3e-3)
torch.testing.assert_close(mean_gpu, mean_ref, atol=5e-3, rtol=3e-3)
torch.testing.assert_close(inv_var_gpu, inv_var_ref, atol=5e-3, rtol=3e-3)

## Using Python Binding APIs

Create input tensor GPU buffers. We use PyTorch to allocate GPU tensors so we can reuse them easily when we calculate reference outputs.

In [ ]:
# input tensor memory, initialize them to random numbers
x_gpu = torch.randn(N, C, H, W, device="cuda", dtype=dtype).to(
    memory_format=torch.channels_last
)
scale_gpu = torch.randn(1, C, 1, 1, device="cuda").to(memory_format=torch.channels_last)
bias_gpu = torch.randn(1, C, 1, 1, device="cuda").to(memory_format=torch.channels_last)
# set epsilon to epsilon_value, allocate on cpu.
eps_cpu = torch.full((1, 1, 1, 1), epsilon_value, dtype=torch.float32, device="cpu")

Create cuDNN graph and cuDNN tensors

In [ ]:
# Create cuDNN graph
graph = cudnn.pygraph(
    handle=handle,
    intermediate_data_type=cudnn.data_type.FLOAT,
    compute_data_type=cudnn.data_type.FLOAT,
)

# create tensor handles with the graph API
x = graph.tensor_like(x_gpu.detach()).set_name("X")
scale = graph.tensor_like(scale_gpu.detach()).set_name("scale")
bias = graph.tensor_like(bias_gpu.detach()).set_name("bias")
epsilon = graph.tensor_like(eps_cpu).set_name("epsilon")

out, mean, inv_var = graph.instancenorm(
    name="instancenorm",
    input=x,
    norm_forward_phase=cudnn.norm_forward_phase.TRAINING,
    scale=scale,
    bias=bias,
    epsilon=epsilon,
)

# enable all outputs
out.set_name("output").set_output(True).set_data_type(dtype)
mean.set_name("mean").set_output(True).set_data_type(dtype)
inv_var.set_name("inv_var").set_output(True).set_data_type(dtype);

Build and execute the graph

In [ ]:
# Build the graph
graph.build([cudnn.heur_mode.A, cudnn.heur_mode.FALLBACK])

# Allocate output tensors
out_gpu = torch.randn(N, C, H, W, device="cuda", dtype=dtype).to(
    memory_format=torch.channels_last
)
mean_gpu = torch.randn(N, C, 1, 1, device="cuda", dtype=dtype).to(
    memory_format=torch.channels_last
)
inv_var_gpu = torch.randn(N, C, 1, 1, device="cuda", dtype=dtype).to(
    memory_format=torch.channels_last
)

# Execute the graph
variant_pack = {
    x: x_gpu,
    scale: scale_gpu,
    bias: bias_gpu,
    epsilon: eps_cpu,
    out: out_gpu,
    mean: mean_gpu,
    inv_var: inv_var_gpu,
}

workspace = torch.empty(graph.get_workspace_size(), device="cuda", dtype=torch.uint8)
graph.execute(variant_pack, workspace)
torch.cuda.synchronize()

Compute reference ouputs using PyTorch and compare with cuDNN graph outputs

In [ ]:
out_ref = torch.nn.functional.instance_norm(
    x_gpu,
    weight=scale_gpu.view(C),
    bias=bias_gpu.view(C),
)
mean_ref = x_gpu.mean(dim=(2, 3), keepdim=True)
inv_var_ref = torch.rsqrt(torch.var(x_gpu, dim=(2, 3), keepdim=True) + epsilon_value)

torch.testing.assert_close(out_gpu, out_ref, rtol=1e-2, atol=1e-2)
torch.testing.assert_close(inv_var_gpu, inv_var_ref, rtol=1e-2, atol=1e-2)
torch.testing.assert_close(mean_gpu, mean_ref, rtol=1e-2, atol=1e-2)

Perform Cleanup

In [ ]:
cudnn.destroy_handle(handle)